In [ ]:
# Import all libraries and modules 
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.documents import Document
from dotenv import load_dotenv

# load the .env file
load_dotenv()

In [ ]:
# Relevant health & wellness documents
docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]


In [ ]:
# Initialization of embedding model
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Create the FAISS vector store from the documents
vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [ ]:
# Create retrievers
similarity_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":5}
)

In [ ]:
# Create llm model
llm = HuggingFaceEndpoint(repo_id="deepseek-ai/DeepSeek-R1-0528", task="text-generation")

# Initialize the Hugging Face chat model
model = ChatHuggingFace(llm=llm)

In [ ]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=model,
    retriever=vector_store.as_retriever(
        search_kwargs={"k":5}
    )
)

In [ ]:
# Query
query = "How to improve energy levels and maintain balance?"

In [ ]:
# Retrievers results
similarity_result = similarity_retriever.invoke(query)
multiquery_result = multiquery_retriever.invoke(query)

In [ ]:
for i , doc in enumerate(similarity_result):
    print(f"Result {i+1}: {doc.page_content}")

In [ ]:
for i , doc in enumerate(multiquery_result):
    print(f"Result {i+1}: {doc.page_content}")